In [ ]:
!pip install openai wikipedia-api tiktoken transformers

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


# Beispiel-Dokumente von Wikipedia

In [ ]:
import tiktoken
import wikipediaapi
from pathlib import Path

In [ ]:
page_name = 'Elvis_Presley'

wiki = wikipediaapi.Wikipedia('LangChain RAG', 'de', extract_format=wikipediaapi.ExtractFormat.WIKI)
text = wiki.page(page_name).text

In [ ]:
len(text)

In [ ]:
from openai import OpenAI
client = OpenAI()

## Übersetzen

In [ ]:
prompt = f'''{text}
Übersetze den folgenden Text ins englische:'''

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )

completion.choices[0].message.content

### Tokens zählen

siehe auch den Tokenizer auf der OpenAI Website: https://platform.openai.com/tokenizer

In [ ]:
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
len(encoding.encode(text))

In [ ]:
# wir schneiden den text nach n-Tokens ab
n = 15000
tokenized_text = encoding.encode(text)
tokenized_text = tokenized_text[:n]

tokenized_text[:10], text[:100]

In [ ]:
encoding.decode([6719])

In [ ]:
text = encoding.decode(tokenized_text)

In [ ]:
prompt

In [ ]:
prompt = f'''{text}

Übersetze den Text vollständig ins englische:'''

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=None
    )

completion.choices[0].message.content

In [ ]:
len(encoding.encode(completion.choices[0].message.content))

#### "finish_reason"

In [ ]:
completion

## Model input / output (Huggingface / OpenSource Modelle)

![llm.png](images/llm.png)

### Wie kommen wir an den "neuen" Token?

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.float32, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

In [ ]:
inputs = tokenizer('''Elvis Aaron Presley (* January 8, 1935 in''', return_tensors="pt", return_attention_mask=False)
inputs['input_ids'].shape

In [ ]:
outputs = model(**inputs)
outputs.logits.shape

In [ ]:
new_token_logits = outputs.logits[:, -1, :]
new_token_logits.shape

In [ ]:
new_token = new_token_logits.argmax(dim=1)
tokenizer.decode(new_token)

### Text generieren mit Huggingface

In [ ]:
outputs = model.generate(**inputs, max_length=20)
text = tokenizer.batch_decode(outputs)[0]
text

### Text generieren ist eine "for loop"

In [ ]:
inputs = tokenizer('''Elvis Aaron Presley (* January 8, 1935 in''', return_tensors="pt", return_attention_mask=False)

for i in range(20):
    # Neuen Token generieren
    outputs = model(**inputs)
    new_token_logits = outputs.logits[:, -1, :]
    new_token = new_token_logits.argmax(dim=1)

    # Anhängen des neuen Tokens an die Inputs
    inputs["input_ids"] = torch.cat((inputs["input_ids"], new_token.unsqueeze(-1)), dim=1)

In [ ]:
tokenizer.decode(inputs["input_ids"][0])